# Capstone 3 Modeling

**The Data Science Method**  

1.   Problem Identification 

2.   Data Wrangling 
  * Data Collection 
   * Data Organization
  * Data Definition 
  * Data Cleaning
 
3.   Exploratory Data Analysis
 * Build data profile tables and plots
        - Outliers & Anomalies
 * Explore data relationships
 * Identification and creation of features

4.   Pre-processing and Training Data Development
  * Create dummy or indicator features for categorical variables
  * Standardize the magnitude of numeric features
  * Split into testing and training datasets
  * Apply scaler to the testing set
  
5.   **Modeling**
  * Fit Models with Training Data Set
  * Review Model Outcomes — Iterate over additional models as needed.
  * Identify the Final Model

6.   Documentation
  * Review the Results
  * Present and share your findings - storytelling
  * Finalize Code 
  * Finalize Documentation

## Data Collection

In [1]:
#load python packages
import os
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
pd.set_option('display.float_format', lambda x: '%.4f' % x) #get rid of scientific notations
import datetime
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import statsmodels.api as sm
import time
import math
from sklearn.metrics import mean_squared_error, mean_absolute_error
from statsmodels.tsa.api import VAR
from IPython.display import Image
%matplotlib inline

In [2]:
# switch to processed data folder
os.chdir('C:\\Users\\tc18f\\Desktop\\springboard\\Capstone Three\\data\\processed\\')
os.getcwd()

'C:\\Users\\tc18f\\Desktop\\springboard\\Capstone Three\\data\\processed'

In [3]:
# load the combined csv file
df = pd.read_csv('combined.csv')
df.head()

,Zipcode,County,Date,Value,Bedrooms
0,94109,San Francisco County,1996-01-31,263374,1
1,90250,Los Angeles County,1996-01-31,184141,1
2,90046,Los Angeles County,1996-01-31,119677,1
3,94501,Alameda County,1996-01-31,119742,1
4,94110,San Francisco County,1996-01-31,221428,1


In [4]:
# check info
df['Date'] = pd.to_datetime(df.Date)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 563732 entries, 0 to 563731
Data columns (total 5 columns):
 #   Column    Non-Null Count   Dtype         
---  ------    --------------   -----         
 0   Zipcode   563732 non-null  int64         
 1   County    563732 non-null  object        
 2   Date      563732 non-null  datetime64[ns]
 3   Value     563732 non-null  int64         
 4   Bedrooms  563732 non-null  int64         
dtypes: datetime64[ns](1), int64(3), object(1)
memory usage: 21.5+ MB


In [5]:
# break the df into 4 of them indicating the number of bedrooms and drop the county and bedrooms column
df1 = df.loc[df['Bedrooms']==1]
df1 = df1.sort_values(['Date','Zipcode'])
df2 = df.loc[df['Bedrooms']==2]
df2 = df2.sort_values(['Date','Zipcode'])
df3 = df.loc[df['Bedrooms']==3]
df3 = df3.sort_values(['Date','Zipcode'])
df4 = df.loc[df['Bedrooms']==4]
df4 = df4.sort_values(['Date','Zipcode'])
df1.tail() # previous when the date wasn't in date time and sort by date will see 2019 in tail()

,Zipcode,County,Date,Value,Bedrooms
140604,95821,Sacramento County,2020-05-31,159237,1
140568,95822,Sacramento County,2020-05-31,301542,1
140598,95825,Sacramento County,2020-05-31,169082,1
140808,96150,El Dorado County,2020-05-31,321712,1
140837,96161,Nevada County,2020-05-31,346110,1


# Preprocess for VARMAX model
set the endog

In [19]:
# define a function that takes in the df, and the number of diff value
def data_trans(df, val_name):
    temp = pd.DataFrame({})
    # add Zipcode as columns and its values to temp
    for zipcode in list(df.Zipcode.unique()):
        temp[zipcode] = list(df[df['Zipcode']==zipcode].Value)
    # add Date column
    temp['Date'] = pd.date_range('1996-01-31', periods=293, freq='M')
    # melt and sort
    temp_melt = pd.melt(temp, id_vars=['Date'], var_name='Zipcode', value_name=val_name)
    temp_sort = temp_melt.sort_values(['Date','Zipcode'])
    # have the Values in int
    temp_sort[val_name] = temp_sort[val_name].astype('int64')
    # set the Date as index
    temp_sort.set_index('Date', inplace=True)
    return temp_sort

In [20]:
df1t = data_trans(df1, 'OneBR')
df2t = data_trans(df2, 'TwoBR')
df3t = data_trans(df3, 'ThreeBR')
df4t = data_trans(df4, 'FourBR')
dft = df1t
dft['TwoBR'] = df2t.TwoBR
dft['ThreeBR'] = df3t.ThreeBR
dft['FourBR'] = df4t.FourBR
dft

,Zipcode,OneBR,TwoBR,ThreeBR,FourBR
Date,,,,,
1996-01-31,90004,139486,184203,123394,133581
1996-01-31,90007,96265,126953,144413,148315
1996-01-31,90012,108276,123930,203276,198879
1996-01-31,90016,103716,142057,158824,180144
1996-01-31,90018,167823,140760,165930,185586
...,...,...,...,...,...
2020-05-31,95821,159237,311446,391195,454362
2020-05-31,95822,301542,324787,327154,389160
2020-05-31,95825,169082,295747,380849,419636


# Modeling

In [16]:
endog.head()

,Zipcode,OneBR,TwoBR,ThreeBR,FourBR
Date,,,,,
1996-01-31,90004,139486,184203,123394,133581
1996-01-31,90007,96265,126953,144413,148315
1996-01-31,90012,108276,123930,203276,198879
1996-01-31,90016,103716,142057,158824,180144
1996-01-31,90018,167823,140760,165930,185586


In [11]:
# set the endog (training data)
endog = dft.loc['1996-1-31':'2015-05-31']
endog.tail()

,Zipcode,OneBR,TwoBR,ThreeBR,FourBR
Date,,,,,
2015-05-31,95821,100591,211218,278828,351642
2015-05-31,95822,122458,198497,222288,265556
2015-05-31,95825,83611,207396,282210,285573
2015-05-31,96150,245861,278758,385019,552845
2015-05-31,96161,279856,377964,516195,889668


In [21]:
# iterate zipcode to subset and model/fit/predict
# create dfs to store values
pred_df1 = pd.DataFrame({'Date':pd.date_range('2015-06-30', periods=60, freq='M')})
pred_df2 = pred_df1
pred_df3 = pred_df1
pred_df4 = pred_df1
# iterate thru zipcode and get the values stored; order=(1,0) trend='n'
start = time.time()
for zipcode in list(dft.Zipcode.unique()):
    warnings.filterwarnings('ignore')
    # get the endog by subsetting the dft with specific zipcode and collumns other thatn zipcode
    endog_sub = endog[endog['Zipcode']==zipcode][[i for i in list(dft.columns)][1:]]
    endog_sub.index.freq = 'M' #set the datetime frequency
    # train/fit the model
    model = sm.tsa.VARMAX(endog_sub, order=(1,0), trend='n')
    result = model.fit(maxiter=1000, disp=False)
    # forecast
    pred = result.predict(start='2015-06-30', end='2020-05-31')
    # append data to dataframes
    pred_df1[zipcode] = list(pred.OneBR)
    pred_df2[zipcode] = list(pred.TwoBR)
    pred_df3[zipcode] = list(pred.ThreeBR)
    pred_df4[zipcode] = list(pred.FourBR)
# stop timer
end = time.time()
fit_time = (end-start)
#check the fit time in min
int(fit_time/60)

C:\Users\tc18f\anaconda3\lib\site-packages\statsmodels\base\model.py:566: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
C:\Users\tc18f\anaconda3\lib\site-packages\statsmodels\base\model.py:566: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
C:\Users\tc18f\anaconda3\lib\site-packages\statsmodels\base\model.py:566: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
C:\Users\tc18f\anaconda3\lib\site-packages\statsmodels\base\model.py:566: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
C:\Users\tc18f\anaconda3\lib\site-packages\statsmodels\base\model.py:566: ConvergenceWarning: Maximu

C:\Users\tc18f\anaconda3\lib\site-packages\statsmodels\base\model.py:566: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
C:\Users\tc18f\anaconda3\lib\site-packages\statsmodels\base\model.py:566: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
C:\Users\tc18f\anaconda3\lib\site-packages\statsmodels\base\model.py:566: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
C:\Users\tc18f\anaconda3\lib\site-packages\statsmodels\base\model.py:566: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
C:\Users\tc18f\anaconda3\lib\site-packages\statsmodels\base\model.py:566: ConvergenceWarning: Maximu

C:\Users\tc18f\anaconda3\lib\site-packages\statsmodels\base\model.py:566: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
C:\Users\tc18f\anaconda3\lib\site-packages\statsmodels\base\model.py:566: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
C:\Users\tc18f\anaconda3\lib\site-packages\statsmodels\base\model.py:566: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
C:\Users\tc18f\anaconda3\lib\site-packages\statsmodels\base\model.py:566: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
C:\Users\tc18f\anaconda3\lib\site-packages\statsmodels\base\model.py:566: ConvergenceWarning: Maximu

C:\Users\tc18f\anaconda3\lib\site-packages\statsmodels\base\model.py:566: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
C:\Users\tc18f\anaconda3\lib\site-packages\statsmodels\base\model.py:566: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
C:\Users\tc18f\anaconda3\lib\site-packages\statsmodels\base\model.py:566: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
C:\Users\tc18f\anaconda3\lib\site-packages\statsmodels\base\model.py:566: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
C:\Users\tc18f\anaconda3\lib\site-packages\statsmodels\base\model.py:566: ConvergenceWarning: Maximu

112

In [22]:
# check what the prediction dataframes look like
display(pred_df1.head(2))
display(pred_df1.tail(2))

,Date,90004,90007,90012,90016,90018,90019,90020,90022,90023,...,95709,95722,95726,95818,95819,95821,95822,95825,96150,96161
0,2015-06-30,1002011.2725,541498.7745,597879.8549,617008.2312,551007.9482,924332.9011,1339044.2596,415507.3155,379589.5985,...,451850.2126,582740.8438,346763.1613,423784.9670,413183.1029,349543.4275,264769.0458,286858.8541,553895.5399,893922.3388
1,2015-07-31,1014829.0791,545819.6974,609720.9977,621136.8446,552350.2515,922263.7144,1351021.9386,420897.6849,384209.3654,...,451587.5002,583002.9547,344301.2055,426344.4757,419733.5358,347498.5027,263962.1392,288166.3484,554841.9955,898259.0044


,Date,90004,90007,90012,90016,90018,90019,90020,90022,90023,...,95709,95722,95726,95818,95819,95821,95822,95825,96150,96161
58,2020-04-30,1959119.8968,761336.9687,960985.1925,1048164.9109,535412.1401,361258.2443,2016833.9872,620931.1737,779169.2993,...,348301.6266,572495.7526,137741.4731,528787.0114,821635.6799,346518.9324,194756.8217,386870.2168,492383.1708,1148633.6912
59,2020-05-31,1978261.3703,764571.2043,957220.7684,1051006.3719,533945.4340,346956.6804,2026487.6614,621436.4217,787506.9216,...,345383.1467,572146.7277,133333.7981,529929.8574,826363.7324,347566.8290,193309.8121,388829.3755,490228.2236,1151006.4314


In [23]:
display(df1.head(2))
display(df1.tail(2))

,Zipcode,County,Date,Value,Bedrooms
22,90004,Los Angeles County,1996-01-31,139486,1
201,90007,Los Angeles County,1996-01-31,96265,1


,Zipcode,County,Date,Value,Bedrooms
140808,96150,El Dorado County,2020-05-31,321712,1
140837,96161,Nevada County,2020-05-31,346110,1


Values are way off.

In [ ]:
# define a function that will revert back the prediction values
def pred_trans(pred_df, val_name):
    temp = pd.DataFrame({}) #create dfs to store values
    restored = pd.DataFrame({'Date':pd.date_range('2015-06-30', periods=60, freq='M')})
    # melt
    temp_melt = pd.melt(restored, id_vars=['Date'], var_name='Zipcode', value_name=val_name)
    # make sure date is datetime
    temp_melt['Date'] = pd.to_datetime(temp_melt.Date)
    # sort
    temp_sort = temp_melt.sort_values(['Date','Zipcode'])
    # have the Values in int so it's easier to read and compared to original values
    temp_sort[val_name] = temp_sort[val_name].astype('int64')
    return temp_sort

In [ ]:
# transform and compile the prediction
pred_dft = pred_trans(pred_df1, df1, 'OneBR')
pred_dft2 = pred_trans(pred_df2, df2, 'TwoBR')
pred_dft3 = pred_trans(pred_df3, df3, 'ThreeBR')
pred_dft4 = pred_trans(pred_df4, df4, 'FourBR')
pred_dft['TwoBR'] = pred_dft2.TwoBR
pred_dft['ThreeBR'] = pred_dft3.ThreeBR
pred_dft['FourBR'] = pred_dft4.FourBR
pred_dft.head()

In [ ]:
# save the restored prediciton to csv file
pred_dft.to_csv('varmax_pred5Dt.csv', index=False)

In [ ]:
# load the transformed data
pred_dft = pd.read_csv('varmax_pred5Dt.csv')
pred_dft['Date'] = pd.to_datetime(pred_dft.Date) # make sure Date column is datetime!
pred_dft.head()

In [ ]:
# let's try plotting the mean/medians of test and pred, we will need to define functions
# define a function that takes in the prediction dataframe and return the medians
def median_calc(data, nobs, val_column):
    p_medians = []
    for i in range(nobs): 
        median = data.sort_values(['Date',val_column]).iloc[240+481*i][val_column]
        p_medians.append(int(median))
    return p_medians
# define a function that takes in the prediction dataframe and return the means
def mean_calc(data, val_column):
    p_means = []
    for i in list(data.Date.unique()): # get the dates to iterate
        mean = data[data['Date']==i][val_column].mean() # get the mean
        p_means.append(int(mean))
    return p_means

In [ ]:
# load the z_avg csv which has zillow's averages
z_avg = pd.read_csv('z_avg.csv')
z_avg['Date'] = pd.to_datetime(z_avg.Date)
z_avg.head()

In [ ]:
# let's compiles the means and medians to form a new dataframe and plot on seaborn
z_df = pd.DataFrame({'Date':pd.date_range('1996-01-31', periods=293, freq='M')})
z_df['z_median_OneBR'] = z_avg.median1
z_df['z_median_TwoBR'] = z_avg.median2
z_df['z_median_ThreeBR'] = z_avg.median3
z_df['z_median_FourBR'] = z_avg.median4
z_df['z_mean_OneBR'] = z_avg.mean1
z_df['z_mean_TwoBR'] = z_avg.mean2
z_df['z_mean_ThreeBR'] = z_avg.mean3
z_df['z_mean_FourBR'] = z_avg.mean4
# do the same for prediction
pred_date_num = pred_dft.Date.nunique()
p_df = pd.DataFrame({'Date':pd.date_range('2015-05-31', periods=pred_date_num, freq='M')})
p_df['p_median_OneBR'] = median_calc(pred_dft, pred_date_num, 'OneBR')
p_df['p_median_TwoBR'] = median_calc(pred_dft, pred_date_num, 'TwoBR')
p_df['p_median_ThreeBR'] = median_calc(pred_dft, pred_date_num, 'ThreeBR')
p_df['p_median_FourBR'] = median_calc(pred_dft, pred_date_num, 'FourBR')
p_df['p_mean_OneBR'] = mean_calc(pred_dft, 'OneBR')
p_df['p_mean_TwoBR'] = mean_calc(pred_dft, 'TwoBR')
p_df['p_mean_ThreeBR'] = mean_calc(pred_dft, 'ThreeBR')
p_df['p_mean_FourBR'] = mean_calc(pred_dft, 'FourBR')

In [ ]:
# melt and we need the following columns for seaborn: Date, measure_type, Value, Bedrooms, Source
z_melt = pd.melt(z_df, ['Date'])
p_melt = pd.melt(p_df, ['Date'])
sea_df = pd.concat([z_melt, p_melt])
sea_df.head()

In [ ]:
# create a list to add the measure type, bedrooms, and sources
measure_list=[]
for i in sea_df.variable:
    if 'median' in i:
        measure_list.append('median')
    else:
        measure_list.append('mean')
# do the same for number of bedrooms
BR_list=[]
for i in sea_df.variable:
    if 'One' in i:
        BR_list.append(1)
    if 'Two' in i:
        BR_list.append(2)
    if 'Three' in i:
        BR_list.append(3)
    if 'Four' in i:
        BR_list.append(4)
# source        
source_list=[]
for i in sea_df.variable:
    if 'z_m' in i:
        source_list.append('zillow')
    else:
        source_list.append('varmax')
# add the columns to sea_df
sea_df['measure_type'] = measure_list
sea_df['Bedrooms'] = BR_list
sea_df['Source'] = source_list
display(sea_df.head(3))
sea_df.tail(3)

In [ ]:
# plot the means using seaborn
sns.set_theme(style="darkgrid")
sea_mean = sea_df[sea_df['measure_type']=='mean'] #subset with only the means
fig, ax = plt.subplots(figsize=(16, 8)) # set the figure size and ax to graph on
sea = sns.lineplot(data=sea_mean, x="Date", y="value", hue="Source", style="Bedrooms", ax=ax)
sea.axes.set_title("Zillow vs VARMAX means",fontsize=18)
sea.set_xlabel("Date",fontsize=14)
sea.set_ylabel("Value (mil)",fontsize=14)
plt.savefig('C:\\Users\\tc18f\\Desktop\\springboard\\Capstone Three\\figures\\Zillow_vs_VARMAX_meansDt.png')

In [ ]:
# plot the median using seaborn
sns.set_theme(style="darkgrid")
sea_mean = sea_df[sea_df['measure_type']=='median'] #subset with only the means
fig, ax = plt.subplots(figsize=(16, 8)) # set the figure size and ax to graph on
sea = sns.lineplot(data=sea_mean, x="Date", y="value", hue="Source", style="Bedrooms", ax=ax)
sea.axes.set_title("Zillow vs VARMAX medians",fontsize=18)
sea.set_xlabel("Date",fontsize=14)
sea.set_ylabel("Value (mil)",fontsize=14)
plt.savefig('C:\\Users\\tc18f\\Desktop\\springboard\\Capstone Three\\figures\\Zillow_vs_VARMAX_mediansDt.png')

The predictions for the first 2 years seems okay, we shall compare the model scores based on each 12 months of data.

In [ ]:
pred_dft.head(2)

In [ ]:
# let's make a test dataframe
def test_data_form(df, BR_num, val_name):
    temp=pd.DataFrame({})
    for zipcode in pred_dft.Zipcode.unique():
        df_sub = df[(df['Zipcode']==zipcode) & (df['Bedrooms']==BR_num)] #subset by zipcode and num of bedroom
        df_sub = df_sub[df_sub['Date']>='2015-05-31'] # get corresponding date range
        temp['Date'] = pd.date_range('2015-05-31', periods=len(df_sub), freq='M')
        temp[zipcode]=list(df_sub.Value)
    # melt
    temp_melt = pd.melt(temp, id_vars=['Date'], var_name='Zipcode', value_name=val_name)
    # make sure date is datetime
    temp_melt['Date'] = pd.to_datetime(temp_melt.Date)
    # sort
    temp_sort = temp_melt.sort_values(['Date','Zipcode'])
    # have the Values in int so it's easier to read and compared to original values
    temp_sort[val_name] = temp_sort[val_name].astype('int64')
    return temp_sort

In [ ]:
# make a compiled dataframe
test = test_data_form(df,1, 'OneBR')
test2 = test_data_form(df,2, 'TwoBR')
test3 = test_data_form(df,3, 'ThreeBR')
test4 = test_data_form(df,4, 'FourBR')
test['TwoBR'] = test2.TwoBR
test['ThreeBR'] = test3.ThreeBR
test['FourBR'] = test4.FourBR
test.head()

In [ ]:
pred_dft.set_index('Date',inplace=True)
test.set_index('Date',inplace=True)

In [ ]:
# calculate the MAEs and store to csv
model_list=[]
MAE_list=[]
for i in range(0,5):
    test_sub = test.iloc[i*481:(1+i)*481*12]
    pred_sub = pred_dft.iloc[i*481:(1+i)*481*12]
    model_list.append('VARMAX5')
    MAE = mean_absolute_error(test_sub, pred_sub)
    MAE_list.append(MAE)

In [ ]:
model_score = pd.DataFrame({
    'Model': model_list,
    'Cycle': ['15to16', '16to17', '17to18','18to19','19to20'],
    'MAE':MAE_list
})
model_score